In [1]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

sc.stop()
conf = pyspark.SparkConf()
conf.set('spark.local.dir', '/Volumes/My Passport')
sc = pyspark.SparkContext(conf=conf)


In [2]:
sqlContext = SQLContext (sc)
sc


<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark.conf.set('spark.sql.shuffle.partitions', 4)
csv_file_path_1 = '/Volumes/My Passport/OpenCorpus/Data/s2-corpus-000'
dataframe = sqlContext.read.json(csv_file_path_1)


In [4]:
#dataframe.schema
dataframe.count()

32140

In [5]:
dataframe_T = dataframe.na.drop()
dataframe_T.count()

31968

In [6]:
dataframe_T.printSchema()
dataframeNew = dataframe_T.select("id", "paperAbstract", "outCitations")
dataframeNew.printSchema()

root
 |-- authors: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- doiUrl: string (nullable = true)
 |-- entities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fieldsOfStudy: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- inCitations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- journalName: string (nullable = true)
 |-- journalPages: string (nullable = true)
 |-- journalVolume: string (nullable = true)
 |-- magId: string (nullable = true)
 |-- outCitations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- paperAbstract: string (nullable = true)
 |-- pdfUrls: array (nullable = true)
 |    |-- element: string (containsNull = 

In [7]:
dataframeNew.show()

+--------------------+--------------------+--------------------+
|                  id|       paperAbstract|        outCitations|
+--------------------+--------------------+--------------------+
|33b237709dbd53953...|                    |                  []|
|062e9c7579adc7312...|Automated medicat...|[c33122bc4d2780bf...|
|771723e7e2d9ed97d...|In the future, th...|[8dc8c738be229d6c...|
|263f70167ba7def67...|                    |                  []|
|9ea1148fa65710387...|                    |                  []|
|a2a482a55978d4363...|                    |                  []|
|984774e366d3d4fcf...|BackgroundCorneal...|[23045299013e8738...|
|0d0467c4cccb0dcf0...|Falsified and sub...|[394cff10a2ac99f7...|
|237e5a3778ccff06c...|Abstract The Ni 0...|[a8a8c4728e4393d7...|
|878a852598fbb063b...|Objectives To exp...|[b858ce153876d8cf...|
|1908dfd37d47b416f...|Through studies o...|                  []|
|a28cf4e1bf123673c...|IntroductionMcCun...|[dd5bf00ab838b888...|
|f73d50b158d8796c6...|   

In [8]:
from pyspark.sql.functions import explode, avg, col

df1 = dataframeNew.select("id", "paperAbstract", explode("outCitations").alias("ref"))
df1.show()

df1.printSchema()

+--------------------+--------------------+--------------------+
|                  id|       paperAbstract|                 ref|
+--------------------+--------------------+--------------------+
|062e9c7579adc7312...|Automated medicat...|c33122bc4d2780bf1...|
|062e9c7579adc7312...|Automated medicat...|90241d37be4b06dd9...|
|062e9c7579adc7312...|Automated medicat...|7d98fde2bdb4e879c...|
|062e9c7579adc7312...|Automated medicat...|2a012d8f4630cd0ba...|
|062e9c7579adc7312...|Automated medicat...|90c7289240281ed73...|
|062e9c7579adc7312...|Automated medicat...|5e266d4c50630b20a...|
|062e9c7579adc7312...|Automated medicat...|ee6e9bd43376f72ae...|
|062e9c7579adc7312...|Automated medicat...|39ae41005ac4a91a4...|
|062e9c7579adc7312...|Automated medicat...|598853073d646f9d8...|
|062e9c7579adc7312...|Automated medicat...|1723e0346fc1de5bb...|
|062e9c7579adc7312...|Automated medicat...|26481340135fb3079...|
|062e9c7579adc7312...|Automated medicat...|94e54a72e5d8c736d...|
|062e9c7579adc7312...|Aut

In [9]:
df2 = df1.na.drop() ## Dropping columns with NULL

In [10]:
df2.count()

293256

In [12]:
## Creating Negative Samples
df2 = df2.limit(300)
df3 = df2.repartition("id")
df1Join = df3.alias("a").join(df1.alias("b")).where('a.ref != b.id')
df1JoinNew = df1Join.select(col("a.id").alias('id1'), col('b.id').alias('id2'),"a.paperAbstract", 
                                                                                col('b.paperAbstract').alias('paperAbstract2')).dropDuplicates()\


In [15]:
from pyspark.sql.functions import lit
df1JoinNew_1 = df1JoinNew.withColumn("label", lit(0))

In [16]:
#df1Join.unpersist()
path = '/Volumes/My Passport/OpenCorpus/data-neg-2.csv'
df1JoinNew_1.repartition(4).write.option("header","true").option("sep",",").mode("overwrite").csv(path)


In [ ]:
## Creating Positive Samples

df1 = df1.repartition("id")
df1Join = df1.alias("a").join(df1.alias("b")).where('a.ref == b.id')
df1Join = df1Join.select(col("a.id").alias('id1'), col('a.ref').alias('id2'),"a.abstract", 
                                                                                col('b.abstract').alias('paperAbstract2')).dropDuplicates()\
    
df1JoinNew_1 = df1Join.withColumn("label", lit(1))
path = '/Volumes/My Passport/OpenCorpus/data-pos-2.csv'
df1JoinNew_1.repartition(4).write.option("header","true").option("sep",",").mode("overwrite").csv(path)
